# Quality Alpha Factors unscreen by value
###### Long term FCF generation
###### Long term ROIC
###### Long term margin characteristics

In [ ]:
#quantopian imports
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume,SimpleBeta, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits

#Python imports
import math
import talib
import numpy as np
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics
from scipy.stats.mstats import winsorize
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from scipy.stats.mstats import gmean
from sklearn.cluster import SpectralClustering
 
from collections import Counter

def preprocess(a):
    """
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[WIN_LIMIT,WIN_LIMIT])
    a= preprocessing.scale(a)
    """
    
    return a

In [ ]:
class Piotroski(CustomFactor):
    inputs = [
        morningstar.operation_ratios.roa,
        morningstar.cash_flow_statement.operating_cash_flow,
        morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities,
        
        morningstar.operation_ratios.long_term_debt_equity_ratio,
        morningstar.operation_ratios.current_ratio,
        morningstar.valuation.shares_outstanding,
        
        morningstar.operation_ratios.gross_margin,
        morningstar.operation_ratios.assets_turnover,
    ]
    window_length = 22
    
    def compute(self, today, assets, out,
                roa, cash_flow, cash_flow_from_ops,
                long_term_debt_ratio, current_ratio, shares_outstanding,
                gross_margin, assets_turnover):
        profit = (
            (roa[-1] > 0).astype(int) +
            (cash_flow[-1] > 0).astype(int) +
            (roa[-1] > roa[0]).astype(int) +
            (cash_flow_from_ops[-1] > roa[-1]).astype(int)
        )
        
        leverage = (
            (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int) +
            (current_ratio[-1] > current_ratio[0]).astype(int) + 
            (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)
        )
        
        operating = (
            (gross_margin[-1] > gross_margin[0]).astype(int) +
            (assets_turnover[-1] > assets_turnover[0]).astype(int)
        )
        
        out[:] = profit + leverage + operating

        
class ROA(CustomFactor):
    window_length = 1
    inputs = [morningstar.operation_ratios.roa]
    
    def compute(self, today, assets, out, roa):
        out[:] = (roa[-1] > 0).astype(int)
        
class ROAChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.operation_ratios.roa]
    
    def compute(self, today, assets, out, roa):
        out[:] = (roa[-1] > roa[0]).astype(int)
        
class CashFlow(CustomFactor):
    window_length = 1
    inputs = [morningstar.cash_flow_statement.operating_cash_flow]
    
    def compute(self, today, assets, out, cash_flow):
        out[:] = (cash_flow[-1] > 0).astype(int)
        
class CashFlowFromOps(CustomFactor):
    window_length = 1
    inputs = [morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities, morningstar.operation_ratios.roa]
    
    def compute(self, today, assets, out, cash_flow_from_ops, roa):
        out[:] = (cash_flow_from_ops[-1] > roa[-1]).astype(int)
        
class LongTermDebtRatioChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.operation_ratios.long_term_debt_equity_ratio]
    
    def compute(self, today, assets, out, long_term_debt_ratio):
        out[:] = (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int)
        
class CurrentDebtRatioChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.operation_ratios.current_ratio]
    
    def compute(self, today, assets, out, current_ratio):
        out[:] = (current_ratio[-1] > current_ratio[0]).astype(int)
        
class SharesOutstandingChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.valuation.shares_outstanding]
    
    def compute(self, today, assets, out, shares_outstanding):
        out[:] = (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)
        
class GrossMarginChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.operation_ratios.gross_margin]
    
    def compute(self, today, assets, out, gross_margin):
        out[:] = (gross_margin[-1] > gross_margin[0]).astype(int)
        
class AssetsTurnoverChange(CustomFactor):
    window_length = 22
    inputs = [morningstar.operation_ratios.assets_turnover]
    
    def compute(self, today, assets, out, assets_turnover):
        out[:] = (assets_turnover[-1] > assets_turnover[0]).astype(int) 

#Not included in Piotroski

class FCFTA_GROWTH(CustomFactor):  
        inputs = [Fundamentals.free_cash_flow,  
                  Fundamentals.total_assets]  
        window_length = 252
        window_safe = True
        def compute(self, today, assets, out, fcf, ta):  
            out[:] = (np.where(fcf[-1]/ta[-1]>fcf[-252]/ta[-252],1,0))

class GM_GROWTH(CustomFactor):  
        inputs = [Fundamentals.gross_margin]  
        window_length = 252
        window_safe = True
        def compute(self, today, assets, out, gm):  
            out[:] = preprocess(np.where(gm[-1]>gm[-252],1,0))

class ATR_GROWTH(CustomFactor):  
        inputs = [Fundamentals.assets_turnover]  
        window_length = 252
        window_safe = True
        def compute(self, today, assets, out, atr):  
            out[:] = preprocess(np.where(atr[-1]>atr[-252],1,0))

class NEQISS(CustomFactor):  
        inputs = [Fundamentals.shares_outstanding]  
        window_length = 252
        window_safe = True
        def compute(self, today, assets, out, so):  
            out[:] = preprocess(np.where(so[-1]-so[-252]<1,1,0))

class GM_GROWTH_2YR(CustomFactor):  
        inputs = [Fundamentals.gross_margin]  
        window_length = 504
        window_safe = True
        def compute(self, today, assets, out, gm):  
            out[:] = preprocess(gmean([gm[-1]+1, gm[-252]+1,gm[-504]+1])-1)

class GM_STABILITY_2YR(CustomFactor):  
        inputs = [Fundamentals.gross_margin]  
        window_length = 504
        window_safe = True
        def compute(self, today, assets, out, gm):  
            out[:] = preprocess(np.std([gm[-1]-gm[-252],gm[-252]-gm[-504]],axis=0)) 

class ROA_GROWTH_2YR(CustomFactor):  
        inputs = [Fundamentals.roa]  
        window_length = 504
        window_safe = True
        def compute(self, today, assets, out, roa):  
            out[:] = (gmean([roa[-1]+1, roa[-252]+1,roa[-504]+1])-1)

class ROIC_GROWTH_2YR(CustomFactor):  
        inputs = [Fundamentals.roic]  
        window_length = 504
        window_safe = True
        def compute(self, today, assets, out, roic):  
            out[:] = (gmean([roic[-1]+1, roic[-252]+1,roic[-504]+1])-1)

class GM_STABILITY_8YR(CustomFactor):  
        inputs = [Fundamentals.gross_margin]  
        window_length = 9
        window_safe = True
        def compute(self, today, assets, out, gm):  
            out[:] = (gm[-8]) 


In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = BOP()
    
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
    
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
    
    outlier_filter1 = alpha_z1.abs() < ZSCORE_FILTER
    zero_filter1 = alpha_weight1.abs() > ZERO_FILTER
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = LINEARREG_SLOPE()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = LINEARREG_INTERCEPT()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = MEDPRICE()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    # ALPHA FACTOR 5
    alpha_factor5 = TRANGE()
    
    # Standardized logic for each input factor after this point
    alpha_w5 = alpha_factor5.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor5.isfinite())
    
    alpha_z5 = alpha_w5.zscore()
    alpha_weight5 = alpha_z5 / 100.0
    
    outlier_filter5 = alpha_z5.abs() < ZSCORE_FILTER
    zero_filter5 = alpha_weight5.abs() > ZERO_FILTER
 
    universe5 = QTradableStocksUS() & \
               outlier_filter5 & \
               zero_filter5
    
    universe =  universe1 & universe2 & universe3 & universe4 & universe5
    
    alpha_weight = alpha_weight2 + alpha_weight3 + alpha_weight4 +alpha_weight5
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'BOP': alpha_weight1,
            'LINEARREG_SLOPE': alpha_weight2,
            'LINEARREG_INTERCEPT': alpha_weight3,
            'MEDPRICE': alpha_weight4,
            'TRANGE': alpha_weight5,
            'sector': sector,
        },
        screen=universe
    )
    return pipe